In [61]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# from sklearn.cluster import KMeans 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.metrics import pairwise_distances

import nltk
import string

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')


# email module has some useful functions
import os, sys, email,re

df = pd.read_csv('arizona_clean.csv')

In [62]:
df.dropna(axis='columns', inplace=True)
df.columns

Index(['Unnamed: 0', 'tweet'], dtype='object')

In [63]:
from sklearn.feature_extraction.text import TfidfVectorizer
data = df['tweet']
tf_idf_vectorizor = TfidfVectorizer(stop_words = 'english',#tokenizer = tokenize_and_stem,
                             max_features = 20000)
tf_idf = tf_idf_vectorizor.fit_transform(data)
tf_idf_norm = normalize(tf_idf)
tf_idf_array = tf_idf_norm.toarray()

In [64]:
pd.DataFrame(tf_idf_array, columns=tf_idf_vectorizor.get_feature_names()).head()

,aaliyah,ab,abagail,abba,abbie,abbot,abbott,abc,abigail,abil,...,zombi,zone,zoo,zookeeperchad,zoolight,zoom,zoommeet,zooud,zuke,zurich
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
from sklearn.cluster import KMeans
sklearn_pca = PCA(n_components = 3)
Y_sklearn = sklearn_pca.fit_transform(tf_idf_array)
kmeans = KMeans(n_clusters=14, max_iter=600)
fitted = kmeans.fit(Y_sklearn)
prediction = kmeans.predict(Y_sklearn)

In [66]:
df['Cluster'] = prediction
df.shape

(3773, 3)

In [67]:
for i in range(df['Cluster'].nunique()):
    print(df[df['Cluster']==i].shape)

(542, 3)
(30, 3)
(64, 3)
(18, 3)
(58, 3)
(487, 3)
(601, 3)
(65, 3)
(1313, 3)
(24, 3)
(37, 3)
(252, 3)
(65, 3)
(217, 3)


In [68]:
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = np.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indices for each cluster
        x_means = np.mean(tf_idf_array[id_temp], axis = 0) # returns average score across cluster
        sorted_means = np.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = tf_idf_vectorizor.get_feature_names()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df)
    return dfs
dfs = get_top_features_cluster(tf_idf_array, prediction, 100)

In [69]:
dfs[0]['features']

0           covid
1     coronavirus
2              az
3            mask
4         arizona
         ...     
95          recov
96        restaur
97           mile
98        sheriff
99         uddudc
Name: features, Length: 100, dtype: object

In [70]:
df_cluster = pd.DataFrame(columns=['Cluster1', 'Cluster2', 'Cluster3', 'Cluster4', 'Cluster5', 'Cluster6', 'Cluster7', 'Cluster8',
                                   'Cluster9', 'Cluster10', 'Cluster11', 'Cluster12', 'Cluster13', 'Cluster14'])

In [71]:
df_cluster[df_cluster.columns[0]] = dfs

In [72]:
df_cluster = pd.DataFrame(columns=['Cluster1', 'Cluster2', 'Cluster3', 'Cluster4', 'Cluster5', 'Cluster6', 'Cluster7', 'Cluster8',
                                   'Cluster9', 'Cluster10', 'Cluster11', 'Cluster12', 'Cluster13', 'Cluster14'])
for i in range(len(dfs)):
    df_cluster[df_cluster.columns[i]] = dfs[i]['features']

In [73]:
df_cluster

,Cluster1,Cluster2,Cluster3,Cluster4,Cluster5,Cluster6,Cluster7,Cluster8,Cluster9,Cluster10,Cluster11,Cluster12,Cluster13,Cluster14
0,covid,kino,pharmaci,coronado,work,social,arizona,home,quarantin,post,az,phoenix,test,work
1,coronavirus,complex,vaccin,photo,job,distanc,quarantin,work,pandem,photo,job,arizona,covid,home
2,az,sport,support,post,home,quarantin,just,az,coronavirus,just,covid,just,free,quarantin
3,mask,site,technicianu,just,appli,day,phoenix,job,socialdistanc,coronado,coach,covid,rd,day
4,arizona,tcobctaizexm,job,district,like,time,tucson,hr,corona,corona,vaccin,quarantin,traffic,look
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,recov,lololol,tcofwnojpid,loluddudcudduddaudcudffenerdlif,tcoszxabohfm,forgot,everi,tcoqlwtzea,better,lockdownnlet,tcoatdbhzvyz,bae,tcojzbzgwno,crew
96,restaur,lolnnnnthisistheend,tcochhpjasfov,lololol,tcopnsgwmqv,hour,best,estim,sunday,lockdownnlalockdown,forward,offic,eas,backyard
97,mile,lolnncovidvacccin,tcotzvghlyuf,loveandlight,tcoqfjhbnxb,saturday,say,latest,say,lockdownndo,representativework,uddudcduddudcd,tcogbnjqjefh,project
98,sheriff,loldisplaytextrang,tcovzfejkfciz,lolnnnnthisistheend,tcotwvqpbiad,activ,museum,return,saturday,lockdownloop,remote,fridayfunni,protect,sasha


In [74]:
df_cluster.to_csv('df_cluster.csv')